In [7]:
### Import necessary libraries
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.common.by import By
from selenium.webdriver import Firefox

data_list_total = []


In [57]:
### Función human behavior TOP
def simulate_human_behavior_top(driver, scroll_min=200, scroll_max=500, min_delay=0.6,max_delay=2.0, verbose=False):
    # Comportamiento simulado de un humano 
    total_height = driver.execute_script("return document.body.scrollHeight")
    current_position = 0

    if verbose:
        print("Iniciando simulación de comportamiento humano...")

    while current_position < total_height:
        scroll_amount = random.randint(scroll_min, scroll_max)
        current_position += scroll_amount
        driver.execute_script(f"window.scrollTo(0, {current_position});")
        
        delay = round(random.uniform(min_delay, max_delay), 2)
        if verbose:
            print(f"Scroll hasta {current_position}px | Esperando {delay}s...")
        time.sleep(delay)
    
    if verbose:
        print("Simulación completada.\n")


In [ ]:
### Función human behavior
def simulate_human_behavior(min_delay=0.8, max_delay=2, verbose=False):
    """Simula comportamiento humano con sleeps aleatorios."""
    delay = round(random.uniform(min_delay, max_delay), 2)
    if verbose:
        print(f"Esperando {delay}s...")
    time.sleep(delay)


In [9]:
# Lista de User-Agents
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7; rv:99.0) Gecko/20100101 Firefox/99.0",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0"
]

def setup_driver():
    """Inicializa el WebDriver de Selenium con configuración de Firefox en modo headless."""
    profile = FirefoxProfile()
    profile.set_preference("general.useragent.override", random.choice(USER_AGENTS))
    profile.set_preference("intl.accept_languages", "es-ES,es")
    firefox_options = Options()
    firefox_options.profile = profile
    #firefox_options.add_argument("--headless")  # Run in headleßss mode
    driver = Firefox(options=firefox_options)
    driver.set_page_load_timeout(10)
    return driver


In [10]:
### Creación del webdriver - Extrar lista de links

all_info = pd.DataFrame()
url = 'https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto%20Rico%2C%20USA&loc=1'
url = 'https://www.martindale.com/search/attorneys/?term=Puerto%20Rico%2C%20USA'

### configuración del webdriver
firefox_options = Options()
    #firefox_options.add_argument("--headless")  # Ejecutar en modo headless 
driver = webdriver.Firefox(options=firefox_options)
wait = WebDriverWait(driver, 10)

cont = 0
list_urls = []
urls_unicos = set()  # Usar un set para almacenar URLs únicas
for i in range(0, 21):
    if i == 0:
        url_actual = url
    
    elif i == 1:
        pass
    else:
        url_actual = url + f"&page={cont}"

    try:
        driver.get(url_actual)
        print(f"accediendo a: {url_actual}")
        time.sleep(2)  # Espera para que la página cargue completamente

        webdriver_wait = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="search-result-container"]/div[5]/div[1]/div[2]/div[1]/div/div'))) 
        attorney_table = driver.find_element(By.XPATH, '//*[@id="search-result-container"]/div[5]/div[1]/div[2]/div[1]/div/div')

        # Extraccion de la lista de abogados
        abogados = attorney_table.find_elements(By.XPATH, './/li[@class="detail_title"]')
        for li in abogados:
            try:
                enlace = li.find_element(By.TAG_NAME, 'a')  # Busca el <a> dentro del <li>
                href = enlace.get_attribute('href')         # Obtiene el atributo href
                
                if href and href not in urls_unicos:
                    list_urls.append(href)
                    urls_unicos.add(href)
                print(href)
            except:
                print("No se encontró <a> en este <li>")
    
    except TimeoutException:
        print("Loading took too much time!")
        driver.quit()
    except Exception as e:
        print(f"Failed to load page due to error: {e}")
        driver.quit()
    cont += 1
driver.quit()


accediendo a: https://www.martindale.com/search/attorneys/?term=Puerto%20Rico%2C%20USA
accediendo a: https://www.martindale.com/search/attorneys/?term=Puerto%20Rico%2C%20USA
accediendo a: https://www.martindale.com/search/attorneys/?term=Puerto%20Rico%2C%20USA&page=2
https://www.martindale.com/attorney/ms-helaine-a-gregory-321442/
https://www.martindale.com/attorney/rafael-l-rovira-arbona-301593668/
https://www.martindale.com/attorney/hector-j-orejuela-davila-168566822/
https://www.martindale.com/attorney/evelyn-rosa-morel-1571797/
https://www.martindale.com/attorney/ramon-mellado-gonzalez-1572913/
https://www.martindale.com/attorney/edgardo-j-hernandez-oharriz-esq-158870123/
https://www.martindale.com/attorney/enrique-g-figueroa-llinas-1571907/
https://www.martindale.com/attorney/alexis-x-garcia-rivera-4671778/
https://www.martindale.com/attorney/jennie-marielle-espada-ocasio-esq-167698736/
https://www.martindale.com/attorney/robert-t-collins-viera-1571688/
https://www.martindale.com/

In [11]:
## Print lista de urls
print(len(list_urls))
list_urls[-10:]  # Muestra los últimos 10 enlaces para verificar



450


['https://www.martindale.com/attorney/santos-ramos-lugo-1573544/',
 'https://www.martindale.com/attorney/louise-shaw-livingston-231132/',
 'https://www.martindale.com/attorney/jarett-blaike-iliff-168913294/',
 'https://www.martindale.com/attorney/ian-marini-biaggi-300364052/',
 'https://www.martindale.com/attorney/juan-a-marques-diaz-1573317/',
 'https://www.martindale.com/attorney/anibal-irizarry-rivera-1573224/',
 'https://www.martindale.com/attorney/nelida-jimenez-velazquez-1571634/',
 'https://www.martindale.com/attorney/brenda-vera-miro-1991152/',
 'https://www.martindale.com/attorney/catherine-catala-lasanta-2501686/',
 'https://www.martindale.com/attorney/omar-cancio-martinez-1573930/']

# Extracción de info de cada link personal

## Caso 1: solo una pagina (sin for)

In [ ]:
### Entrar a la URL - una sola

# Usar Selenium solo para verificar que el elemento esté presente
firefox_options = Options()
#firefox_options.add_argument("--headless")  # Ejecutar en modo headless 
driver = webdriver.Firefox(options=firefox_options)
wait = WebDriverWait(driver, 10)
driver.get(url)
info = {}
practice_areas = set()  # Usar un set para evitar duplicados

try:
    
    wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, 'body > div.off-canvas-wrap > div > div.attorney-profile-content.profile-content')
    ))
    # Extraer el HTML usando requests y procesar con BeautifulSoup
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    profile_content = soup.select_one('div.attorney-profile-content.profile-content')
    if profile_content:
        info["url"] = url
        info["name"] = profile_content.find('h1').get_text(strip=True)
        #print(f"Nombre del abogado: {info['name']}")
        h2_title = profile_content.find_all('h2')
        
        # Adiccion de info
        start_extracting = False
        for h in h2_title[:]:
            # Crear los keys con la info de los abogados
            key = h.get_text(strip=True)
            if "Areas of Practice" in key:
                key ="Areas of Practice"
                info[key] = ""
            else: 
                info[key] = ""

            
            #if list(info.keys())[-1] == "Biography":
            #    start_extracting = True
            #    #print("Iniciando extracción de información...")
            #if not start_extracting:
            #    continue  # Saltar mientras no se haya encontrado "Biography"

            # Obtener el siguiente elemento hermano del h2
            #content_h2 = h.find_next_sibling(class= "toggle-area_content")
            #print(f"\nKey: {h.get_text(strip=True)}")
            content_h2 = h.find_next_sibling(lambda tag: tag.name == 'div' and 'toggle-area__content' in tag.get('class', []))
            
            if content_h2:
                child_text = content_h2.find('div', class_='truncate-text')
                row_collapse = content_h2.find('div', class_='row collapse')
                rows_collapse = content_h2.find_all(
                    lambda tag: tag.name == 'div' and 'row' in tag.get('class', []) 
                    and 'collapse' in tag.get('class', [])
                    and 'experience-section' in tag.get('class', [])
                    and 'clearfix' in tag.get('class', []))
                firm_offices = content_h2.find('ul', id='firmOffices')
                #location_div = soup.find('div', class_='office-address')
                #print(len(location_info))

                if child_text:
                    child_ver = child_text.get_text(strip=True)
                    info[key] = child_ver
                    #print(info[h.get_text(strip=True)])
                if row_collapse:
                    ul_info = row_collapse.find('ul')
                    for line in ul_info:
                        line_text = line.get_text(strip=True)
                if rows_collapse:
                    for text in rows_collapse:
                        text_info = text.get_text(strip=True)
                        for line in text_info.split('\n'):
                            if ":" in line:
                                line = line.strip()
                                k_education, v_education = map(str.strip, line.split(":", 1))
                                info[k_education] = v_education
                                #print(f"{k_education}")
                            else:
                                info[key] = line.strip()
                                #print(line.strip())
                if firm_offices:
                    location_items = firm_offices.find_all('div', class_='office-address')
                    info[key] = location_items[0].text.strip()
            else:
                del info[f"{h.get_text(strip=True)}"] # Eliminar los elementos vacios
    else:
        print("No se encontró el perfil del abogado en el HTML.")
    driver.quit()


except NoSuchElementException:
    print(f"Element not found on the page: {url}")
except TimeoutException:
    print(f"Timeout while waiting for the page to load: {url}")


In [ ]:
df = pd.DataFrame([info])
df


## Caso 2: Recorrido todos los links (for)

In [13]:
def scrape_abogados(lista_urls):
    options = Options()
    # options.add_argument("--headless")  # Descomenta si no quieres abrir la ventana
    
    profile = webdriver.FirefoxProfile()
    profile.set_preference("general.useragent.override", random.choice([
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:119.0) Gecko/20100101 Firefox/119.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13.4; rv:115.0) Gecko/20100101 Firefox/115.0",
        "Mozilla/5.0 (X11; Linux x86_64; rv:116.0) Gecko/20100101 Firefox/116.0"
    ]))
    profile.set_preference("dom.webdriver.enabled", False)
    resultados = []
    count = 1
    for url in lista_urls:
        #driver = webdriver.Firefox(options=options)
        driver = setup_driver()
        wait = WebDriverWait(driver, 15)
        info = {}
        try:
            simulate_human_behavior(1.0, 2.0, verbose=False)
            driver.get(url)
            simulate_human_behavior(0.8, 2.0, verbose=False)

            print(f"accediendo a {count}: {url}")
            wait.until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'body > div.off-canvas-wrap > div > div.attorney-profile-content.profile-content')
            ))

            simulate_human_behavior(1.0, 2.0, verbose=False)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            profile_content = soup.select_one('div.attorney-profile-content.profile-content')

            if not profile_content:
                print(f"Perfil no encontrado para {url}")
                continue

            info["url"] = url
            info["name"] = profile_content.find('h1').get_text(strip=True)
            h2_title = profile_content.find_all('h2')

            for h in h2_title:
                simulate_human_behavior(0.1, 0.5)
                key = h.get_text(strip=True)
                if "Areas of Practice" in key:
                    key = "Areas of Practice"
                info[key] = ""

                content_h2 = h.find_next_sibling(
                    lambda tag: tag.name == 'div' and 'toggle-area__content' in tag.get('class', [])
                )
                if not content_h2:
                    del info[key]
                    continue

                # Caso 1: contenido en div.truncate-text
                child_text = content_h2.find('div', class_='truncate-text')
                if child_text:
                    info[key] = child_text.get_text(strip=True)

                # Caso 2: ul con areas
                row_collapse = content_h2.find('div', class_='row collapse')
                if row_collapse:
                    ul_info = row_collapse.find('ul')
                    if ul_info:
                        items = [li.get_text(strip=True) for li in ul_info.find_all('li')]
                        info[key] = ', '.join(items)

                # Caso 3: datos tipo educación, experiencia, etc.
                rows_collapse = content_h2.find_all(
                    lambda tag: tag.name == 'div' and 
                    all(cls in tag.get('class', []) for cls in ['row', 'collapse', 'experience-section', 'clearfix'])
                )
                for text in rows_collapse:
                    text_info = text.get_text(strip=True)
                    for line in text_info.split('\n'):
                        if ':' in line:
                            k, v = map(str.strip, line.split(":", 1))
                            info[k] = v
                        else:
                            info[key] = line.strip()

                # Caso 4: dirección y contacto
                firm_offices = content_h2.find('ul', id='firmOffices')
                if firm_offices:
                    location_items = firm_offices.find_all('div', class_='office-address')
                    if location_items:
                        info["Location"] = location_items[0].get_text(strip=True)
            if info:
                resultados.append(info)
            count += 1 
            driver.quit()
        except (NoSuchElementException, TimeoutException) as e:
            print(f"Error procesando {url}: {str(e)}")
            driver.quit()
            break
        
    driver.quit()
    # Resultado ya es una lista de diccionarios
    return resultados


In [14]:
# lista de urls 
len(list_urls)
test_list = list_urls[0:5]
# Corridas por batches
list_1 = list_urls[6:100]
list_2 = list_urls[101:300]
list_3 = list_urls[301:]


In [16]:
# Ejecutar el scraper
datos = scrape_abogados(list_urls)
len(datos)
data_list_total.append(datos)


accediendo a 1: https://www.martindale.com/attorney/ms-helaine-a-gregory-321442/
accediendo a 2: https://www.martindale.com/attorney/rafael-l-rovira-arbona-301593668/
accediendo a 3: https://www.martindale.com/attorney/hector-j-orejuela-davila-168566822/
accediendo a 4: https://www.martindale.com/attorney/evelyn-rosa-morel-1571797/
accediendo a 5: https://www.martindale.com/attorney/ramon-mellado-gonzalez-1572913/
accediendo a 6: https://www.martindale.com/attorney/edgardo-j-hernandez-oharriz-esq-158870123/
accediendo a 7: https://www.martindale.com/attorney/enrique-g-figueroa-llinas-1571907/
accediendo a 8: https://www.martindale.com/attorney/alexis-x-garcia-rivera-4671778/
accediendo a 9: https://www.martindale.com/attorney/jennie-marielle-espada-ocasio-esq-167698736/
accediendo a 10: https://www.martindale.com/attorney/robert-t-collins-viera-1571688/
accediendo a 11: https://www.martindale.com/attorney/alexandre-a-afanassiev-3711880/
accediendo a 12: https://www.martindale.com/attor

In [18]:
print(len(data_list_total))
df = pd.DataFrame(datos)
df.to_excel("datos_abogados.xlsx", index=False)
df.tail()



2


,url,name,Biography,Areas of Practice,Education & Credentials,Contact Information,University Attended,Law School Attended,Year of First Admission,Admission,...,"Other Practice Areas in Rio Piedras, Puerto Rico","Other Practice Areas in Dorado, Puerto Rico","Other Practice Areas in Arecibo, Puerto Rico","Other Practice Areas in Bayamon, Puerto Rico","Other Practice Areas in Caguas, Puerto Rico","Other Practice Areas in Cayey, Puerto Rico","Other Practice Areas in Carolina, Puerto Rico","Other Practice Areas in Trujillo Alto, Puerto Rico","Other Practice Areas in Aguadilla, Puerto Rico","Other Practice Areas in Isabela, Puerto Rico"
445,https://www.martindale.com/attorney/anibal-iri...,Aníbal Irizarry-Rivera,NaN,"Wage and Hour Legislation, Occupational Safety...",,NaN,NaN,"New York University, LL.M., 1967; University o...",1967,"1967, Puerto Rico",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
446,https://www.martindale.com/attorney/nelida-jim...,Nélida Jiménez VelÁzquez,NaN,Construction Law,,NaN,"University of Puerto Rico, B.A., 1990","University of Puerto Rico, J.D., 1993",1994,"1994, Puerto Rico",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447,https://www.martindale.com/attorney/brenda-ver...,Brenda Vera Miro,NaN,"Corporate Law, Insurance, Real Estate, Banking...",,NaN,"Pontifical Catholic University of Puerto Rico,...","Pontifical Catholic University of Puerto Rico,...",1997,"1997, Puerto Rico",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448,https://www.martindale.com/attorney/catherine-...,Catherine Catalá Lasanta,NaN,"Civil Litigation, Federal Litigation, Labor Law",,NaN,"Wesleyan University, B.A., 1996","University of Puerto Rico, School of Law,, J.D...",2001,"2001, Puerto Rico; 2002, U.S. District Court, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
449,https://www.martindale.com/attorney/omar-canci...,Omar Cancio Martínez,NaN,"Tax Law, Corporate Law, Financing Law",,NaN,"University of Puerto Rico, B.B.A., 1977","Inter-American University of Puerto Rico, J.D....",1980,"1980, Puerto Rico; 1981, U.S. District Court f...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##